In [ ]:
# {“op”:”u”,
# “ts”:178798798,
# “before”:
#     {“id”: 1, “name”: “ayush”, “age”: 12},
# “after”:
#     {“id”: 1, “name”: “ayush”, “age”: 13},

# }
# op: u/c
# Dipesh Chandra Maurya
# 4:23 PM
# {“op”:”c”,
# “ts”:178798797,
# “before”:
#     {},
# “after”:
#     {“id”: 1, “name”: “ayush”, “age”: 11}
# }
# primary key: id
# Dipesh Chandra Maurya
# 4:37 PM
# created_at, updated_at

def get_df():
    return ""
def get_cur_data(updated_data):
    
    return ""

def store(final_data):
    final_data.show()
    # store the partition data .partition_by()
    

# data
data = get_df()


# data.op == u (update)
updated_data = data.filter(col("op") == "u").order_by(col("ts"))

# data.op == c (create)
created_data = data.filter(col("op") == "c")
curr_data = get_cur_data(updated_data).union(created_data.select(col("created_data.after.id"), col("created_data.after.age").as("age"), col("created_data.after.name")))


# update the curr_data
final_data = updated_data.join(curr_data, (col("updated_data.after.id") == col("curr_data.id")))\
    .select(col("created_data.id"), coalsec(col("updated_data.after.name"), col("curr_data.name")), coalsec(col("updated_data.after.age"), col("curr_data.age")))


# # add the created data

# final_data= final_data.union(created_data.select(col("created_data.after.id"), col("created_data.after.age").as("age"), col("created_data.after.name")))




# store

store(final_data)










